In [2]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [3]:
userlog = pd.read_csv('user_logs.csv')

In [4]:
# Set datetime object as the index
userlog = userlog.set_index(userlog.datetime)
# Drop the original datetime column
userlog.drop(columns='datetime',inplace=True)

In [5]:
# Set 'datetime' as a datetime index.
userlog.index = pd.to_datetime(userlog.index)

In [6]:
userlog.head()

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,,
2018-01-26 09:55:03,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,/,1,97.105.19.61
2018-01-26 09:56:02,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,java-ii,1,97.105.19.61
2018-01-26 09:56:05,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,java-ii/object-oriented-programming,1,97.105.19.61
2018-01-26 09:56:06,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,slides/object_oriented_programming,1,97.105.19.61
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,javascript-i/conditionals,2,97.105.19.61


## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [7]:
# Drop 'deleted_at' column
userlog.drop(columns='deleted_at', inplace=True)

In [8]:
userlog.dropna(inplace=True)

In [9]:
datascience = userlog[userlog.program_id == 3]

In [12]:
webdev = userlog[userlog.program_id != 3]

## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [13]:
datascience.path.value_counts()

/                                           8358
search/search_index.json                    2203
classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
                                            ... 
python/custom-sorting-functions                1
imports                                        1
java-i/console-io                              1
appendix/univariate_regression_in_excel        1
6-regression/8-Project                         1
Name: path, Length: 682, dtype: int64

In [14]:
webdev.path.value_counts()

/                                                               37496
javascript-i                                                    18193
toc                                                             17580
search/search_index.json                                        15331
java-iii                                                        13162
                                                                ...  
4-stats/2.7-correlation                                             1
4-python/error-handling                                             1
10-anomaly-detection/isolation-forests                              1
10-anomaly-detection/time-series-anomaly-detection-part-3           1
appendix/professional-development/post-interview-review-form        1
Name: path, Length: 2054, dtype: int64

### Takeaway 
- We assume that / has the most hits because it is access to the main site 37496 requests for webdev and 8358 for data science
- The lesson search/search_index.json attracts mmore traffic for the Data  science courses 2203 requests
- The lesson javascript-i   attracts mmore traffic for the web development courses 18193 requests


## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


In [29]:
datascience.groupby(['cohort_name']).path.value_counts()

cohort_name  path                                    
Bayes        /                                           1967
             1-fundamentals/modern-data-scientist.jpg     650
             1-fundamentals/AI-ML-DL-timeline.jpg         648
             1-fundamentals/1.1-intro-to-data-science     640
             search/search_index.json                     588
                                                         ... 
Florence     stats/simulation                               1
             storytelling/creating-charts                   1
             storytelling/overview                          1
             timeseries/modeling-lesson2                    1
             where                                          1
Name: path, Length: 1763, dtype: int64

In [22]:
datascience.cohort_name.value_counts()

Darden      32015
Bayes       26538
Curie       21581
Easley      14715
Florence     8562
Name: cohort_name, dtype: int64

In [31]:
datascience.head()

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2019-08-20 09:39:58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,466,97.105.19.58
2019-08-20 09:39:59,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,467,97.105.19.58
2019-08-20 09:39:59,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,468,97.105.19.58
2019-08-20 09:40:02,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,469,97.105.19.58
2019-08-20 09:40:08,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,470,97.105.19.58


In [44]:
datascience[datascience.cohort_name =='Bayes'].groupby('path').count().cohort_name.nlargest(10)

path
/                                                    1967
1-fundamentals/modern-data-scientist.jpg              650
1-fundamentals/AI-ML-DL-timeline.jpg                  648
1-fundamentals/1.1-intro-to-data-science              640
search/search_index.json                              588
6-regression/1-overview                               521
10-anomaly-detection/AnomalyDetectionCartoon.jpeg     387
10-anomaly-detection/1-overview                       384
6-regression/5.0-evaluate                             333
5-stats/3-probability-distributions                   320
Name: cohort_name, dtype: int64

In [47]:
cohort_list_ds = list(datascience.cohort_name.unique())

In [48]:
for cohort in cohort_list_ds:
    print(cohort)
    print(datascience[datascience.cohort_name == cohort].groupby('path').count().cohort_name.nlargest(5))
    print('====================================================================')

Bayes
path
/                                           1967
1-fundamentals/modern-data-scientist.jpg     650
1-fundamentals/AI-ML-DL-timeline.jpg         648
1-fundamentals/1.1-intro-to-data-science     640
search/search_index.json                     588
Name: cohort_name, dtype: int64
Curie
path
/                                           1712
6-regression/1-overview                      595
search/search_index.json                     538
1-fundamentals/modern-data-scientist.jpg     467
1-fundamentals/AI-ML-DL-timeline.jpg         465
Name: cohort_name, dtype: int64
Darden
path
/                                           2980
classification/overview                     1109
classification/scale_features_or_not.svg     943
sql/mysql-overview                           774
search/search_index.json                     664
Name: cohort_name, dtype: int64
Easley
path
/                                                                1115
classification/scale_features_or_not.svg             

In [49]:
cohort_list_wd = list(webdev.cohort_name.unique())

In [50]:
for cohort in cohort_list_wd:
    print(cohort)
    print(webdev[webdev.cohort_name == cohort].groupby('path').count().cohort_name.nlargest(5))
    print('====================================================================')

Hampton
path
/           210
java-iii     57
appendix     55
java-i       46
java-ii      46
Name: cohort_name, dtype: int64
Teddy
path
/               1828
spring           744
java-iii         732
mysql            638
javascript-i     615
Name: cohort_name, dtype: int64
Sequoia
path
/                                  630
spring                             364
spring/fundamentals/views          207
javascript-i                       202
spring/fundamentals/controllers    190
Name: cohort_name, dtype: int64
Arches
path
/                626
javascript-i     294
html-css         215
javascript-ii    204
spring           192
Name: cohort_name, dtype: int64
Niagara
path
spring        65
index.html    63
/             37
java-iii      29
mysql         24
Name: cohort_name, dtype: int64
Pinnacles
path
/                149
javascript-i      69
javascript-ii     58
html-css          56
spring            56
Name: cohort_name, dtype: int64
Quincy
path
/                                           

path
/    1
Name: cohort_name, dtype: int64
Hyperion
path
/               1365
toc             1022
javascript-i     910
java-iii         701
java-ii          688
Name: cohort_name, dtype: int64
Bash
path
/                           772
search/search_index.json    660
toc                         404
javascript-i                374
java-iii                    324
Name: cohort_name, dtype: int64
Jupiter
path
toc                         1866
/                           1696
search/search_index.json     998
javascript-i                 926
java-iii                     795
Name: cohort_name, dtype: int64
Kalypso
path
/               939
javascript-i    631
html-css        495
toc             495
jquery          470
Name: cohort_name, dtype: int64
Luna
path
javascript-i                                                                 676
/                                                                            491
search/search_index.json                                                    

## Takeaway
- For data science, the Darden cohort accessed classification/overview lesson more that other data science cohorts with  1109 requests
- For web Dev, 

## 3 Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?


In [60]:
active_ds_students = datascience[datascience.index < datascience.end_date]

In [61]:
active_wd_students = webdev[webdev.index < webdev.end_date]

In [62]:
active_wd_students

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/conditionals,2,97.105.19.61
2018-01-26 09:56:41,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/loops,2,97.105.19.61
2018-01-26 09:56:46,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/conditionals,3,97.105.19.61
2018-01-26 09:56:48,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/functions,3,97.105.19.61
2018-01-26 09:56:59,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/loops,2,97.105.19.61
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:36:09,135,Marco,#marco,2021-01-25,2021-07-19,2021-01-20 21:31:11,2021-01-20 21:31:11,2,jquery/personal-site,869,136.50.98.51
2021-04-21 16:36:34,138,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2,html-css/css-ii/bootstrap-grid-system,948,104.48.214.211
2021-04-21 16:37:48,134,Luna,#luna,2020-12-07,2021-06-08,2020-12-07 16:58:43,2020-12-07 16:58:43,2,java-iii,834,67.11.50.23


In [65]:
# data science students with least access to the curriculum 
ds_user_id = list(active_ds_students.user_id.unique())


In [66]:
for user in ds_user_id:
    print(user)
    print(active_ds_students[active_ds_students.user_id == user].groupby('path').count().user_id.nsmallest(25))
    print('====================================================================')

466
path
10-anomaly-detection/2-detecting-through-probability    1
10-anomaly-detection/4-detecting-with-clustering        1
10-anomaly-detection/exercises                          1
11-nlp/topic-modeling                                   1
12-distributed-ml/1-overview                            1
12-distributed-ml/5-explore                             1
12-distributed-ml/6-ml-with-spark                       1
12-distributed-ml/spark-execution-diagram.svg           1
12-distributed-ml/spark-local-mode.svg                  1
13-advanced-topics/2-cross-validation                   1
13-advanced-topics/3.6-html-forms                       1
13-advanced-topics/3.8-json-responses                   1
3-sql/6.1-clauses-overview                              1
3-sql/9.1-relationships-overview                        1
4-python/7.4.1-pandas-overview                          1
6-regression/built-in-datasets                          1
6-regression/ridge-regression                           1
7-cla

path
1-fundamentals/2.1-excel-overview                         1
1-fundamentals/2.2-excel-functions                        1
2-storytelling/1-overview                                 1
2-storytelling/2.1-understand                             1
2-storytelling/2.2-create                                 1
2-storytelling/2.3-refine                                 1
3-sql/6.1-clauses-overview                                1
3-sql/6.2-where                                           1
3-sql/6.3-order-by                                        1
3-sql/6.4-limit                                           1
3-sql/9.1-relationships-overview                          1
3-sql/9.3-joins                                           1
6-regression/1-overview                                   1
advanced-topics/json-responses                            1
anomaly-detection/time-series-anomaly-detection-part-2    1
appendix/cli/3-file-paths                                 1
appendix/cli/7-more-topics         

In [67]:
# webdev students with the least access to the curriculum 
wd_user_id = list(active_wd_students.user_id.unique())


In [68]:
 for user in wd_user_id:
    print(user)
    print(active_wd_students[active_wd_students.user_id == user].groupby('path').count().user_id.nsmallest(25))
    print('====================================================================')

2
path
appendix/angular/templating                                    1
appendix/further-reading/java/intellij-tomcat-configuration    1
appendix/further-reading/javascript/array-splice               1
appendix/further-reading/spring/devtools-configuration         1
appendix/further-reading/spring/pagination                     1
appendix/git/intellij-intro                                    1
html-css/css-i/introduction                                    1
html-css/css-ii/bootstrap-grid-system                          1
prework/cli/04-navigating-the-filesystem                       1
prework/cli/05-creating-files-and-directories                  1
prework/cli/06-moving-files                                    1
appendix/git/branching                                         2
examples/java/MoviesArray.java                                 2
examples/javascript/ajax-store.html                            2
examples/javascript/defuse-the-bom.html                        2
mysql/clauses/limi

path
appendix/code-standards                                                         1
appendix/code-standards/mysql                                                   1
appendix/git/merge-conflict-demo                                                1
appendix/git/working-in-teams                                                   1
appendix/postwork                                                               1
appendix/postwork/basic-challenges                                              1
appendix/postwork/locations                                                     1
appendix/postwork/memory-game                                                   1
appendix/postwork/pdf-parser                                                    1
content/git                                                                     1
content/git/adding-files.html                                                   1
content/git/creating-a-repository.html                                          1
content/git

228
path
appendix/angular/dependencies                                  1
appendix/angular/models                                        1
appendix/angular/templating                                    1
appendix/code-standards/html-css                               1
appendix/extra-challenges/basic-challenges                     1
appendix/further-reading/java/intellij-tomcat-configuration    1
appendix/further-reading/spring/devtools-configuration         1
appendix/further-reading/spring/security-use-cases             1
appendix/git/cli                                               1
content/php_i                                                  1
content/php_i/gitbook/images/favicon.ico                       1
examples/bootstrap-grid/assets.zip                             1
examples/css/bootstrap-grids.html                              1
examples/css/login-form.html                                   1
examples/css/media-queries.html                                1
examples/java/Mo

path
appendix                                                       1
appendix/further-reading/java/intellij-tomcat-configuration    1
appendix/git/branching                                         1
appendix/git/intellij                                          1
appendix/git/intellij-intro                                    1
examples/css/media-queries.html                                1
examples/css/messages.html                                     1
examples/javascript/blog.json                                  1
examples/javascript/dom-query-js.html                          1
examples/javascript/inventory.json                             1
html-css/css-ii                                                1
java-i/console-io                                              1
javascript-i/bom-and-dom                                       1
javascript-i/bom-and-dom/dom-events                            1
jquery/ajax                                                    1
jquery/events       

path
appendix/code-standards/javascript                                      1
appendix/extra-challenges/beef-up-your-blog                             1
appendix/extra-challenges/css-practice                                  1
appendix/extra-challenges/memory-game                                   1
appendix/git/branching                                                  1
appendix/git/cli                                                        1
appendix/git/merge-conflict-demo                                        1
content/php_i/gitbook/images/favicon.ico                                1
content/php_i/intro-via-interactive-shell/gitbook/images/favicon.ico    1
examples/css/messages.html                                              1
examples/java/MoviesArray.java                                          1
examples/javascript/ajax-store.html                                     1
examples/javascript/blog.json                                           1
examples/javascript/defuse-the-bo

...

path
appendix                                                 1
appendix/further-reading/javascript/hoisting             1
appendix/further-reading/javascript/this                 1
appendix/pair-programming                                1
appendix/professional-development/professionalism-101    1
examples/bootstrap-grid/assets.zip                       1
examples/css/css_selectors.html                          1
examples/css/flex-form.zip                               1
examples/css/messages.html                               1
examples/java/MoviesArray.java                           1
examples/javascript/ajax-store.html                      1
examples/javascript/blog.json                            1
examples/javascript/inventory.json                       1
html-css/css-i/css-resets                                1
mysql/extra-exercises                                    1
spring/extra-features                                    1
spring/extra-features/form-validation              

path
appendix/capstone-workbook/sw-project-planning            1
appendix/further-reading/spring/devtools-configuration    1
appendix/git/branching                                    1
appendix/pair-programming                                 1
appendix/professional-development/linkedin-tips           1
capstone/repo-doc                                         1
capstone/workbook                                         1
examples/css/flex-form.zip                                1
examples/css/flexbox-cards.zip                            1
examples/css/media-queries.html                           1
examples/css/messages.html                                1
examples/css/twitter.html                                 1
examples/java/MoviesArray.java                            1
examples/javascript/ajax-store.html                       1
examples/javascript/blog.json                             1
examples/javascript/defuse-the-bom.html                   1
examples/javascript/dom-query-js.ht

path
appendix/code-standards                                   1
appendix/code-standards/java                              1
appendix/email-setup                                      1
appendix/extra-challenges/beef-up-your-blog               1
appendix/extra-challenges/java-file-io-extras             1
appendix/extra-challenges/website-status-page             1
appendix/further-reading/spring/devtools-configuration    1
appendix/git/branching                                    1
appendix/git/cli                                          1
appendix/git/intellij-intro                               1
appendix/git/working-in-teams                             1
appendix/offer-letters                                    1
appendix/professional-development/cover-letter            1
appendix/professional-development/email-setup             1
appendix/professional-development/offer-letters           1
appendix/professional-development/virtual-meetings        1
examples/javascript/ajax-store.html

path
appendix/angular/controllers                                     1
appendix/further-reading/html-css/flexbox-additional-concepts    1
appendix/further-reading/javascript/array-splice                 1
appendix/git/cli                                                 1
appendix/professional-development/t-block-resume                 1
examples/bootstrap-grid/assets.zip                               1
examples/css/bootstrap-grids.html                                1
examples/css/flex-form.zip                                       1
examples/css/flexbox-cards.zip                                   1
examples/css/login-form.html                                     1
examples/javascript/defuse-the-bom.html                          1
examples/javascript/inventory.json                               1
html-css/css-i                                                   1
mysql/clauses                                                    1
mysql/intellij                                           

path
appendix/further-reading/html-css/flexbox-additional-concepts    1
examples/css/flexbox-cards.zip                                   1
html-css/introduction                                            1
javascript-i/bom-and-dom                                         1
javascript-i/bom-and-dom/bom                                     1
javascript-i/coffee-project                                      1
javascript-ii                                                    1
examples/css/flex-form.zip                                       2
examples/css/messages.html                                       2
examples/css/twitter.html                                        2
html-css/css-ii/bootstrap-grid-system                            2
javascript-i/objects                                             2
javascript-i/testing-user-functions                              2
examples/css/css_selectors.html                                  3
examples/css/media-queries.html                          

## Takeaway

## 4 Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [73]:
userlog.ip.value_counts()

97.105.19.58       268648
97.105.19.61        60530
192.171.117.210      9124
71.150.217.33        6791
76.185.145.231       4754
                    ...  
45.56.151.121           1
193.37.252.68           1
107.77.66.54            1
99.203.27.219           1
96.8.130.209            1
Name: ip, Length: 5200, dtype: int64

In [76]:
webdev[webdev.ip == '97.105.19.58']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2018-07-16 07:51:57,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1,javascript-ii/promises,211,97.105.19.58
2018-07-16 07:58:03,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1,javascript-ii/map-filter-reduce,211,97.105.19.58
2018-07-16 07:59:50,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1,javascript-ii/promises,211,97.105.19.58
2018-07-16 08:07:37,14,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,1,index.html,53,97.105.19.58
2018-07-16 08:07:41,14,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,1,javascript-ii,53,97.105.19.58
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-23 15:10:09,57,Ganymede,#ganymede,2020-03-23,2020-08-20,2020-03-23 17:52:16,2020-03-23 17:52:16,2,toc,625,97.105.19.58
2020-03-23 15:10:18,57,Ganymede,#ganymede,2020-03-23,2020-08-20,2020-03-23 17:52:16,2020-03-23 17:52:16,2,toc,623,97.105.19.58
2020-03-23 15:20:39,57,Ganymede,#ganymede,2020-03-23,2020-08-20,2020-03-23 17:52:16,2020-03-23 17:52:16,2,/,627,97.105.19.58


In [77]:
webdev[webdev.ip == '97.105.19.61']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2018-01-26 09:55:03,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,/,1,97.105.19.61
2018-01-26 09:56:02,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,1,97.105.19.61
2018-01-26 09:56:05,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii/object-oriented-programming,1,97.105.19.61
2018-01-26 09:56:06,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides/object_oriented_programming,1,97.105.19.61
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/conditionals,2,97.105.19.61
...,...,...,...,...,...,...,...,...,...,...,...
2018-07-13 16:30:55,18,Pinnacles,#pinnacles,2017-03-27,2017-07-20,2017-03-28 00:33:12,2017-03-28 00:33:12,2,slides/object_oriented_programming,26,97.105.19.61
2018-07-13 16:38:26,23,Ulysses,#ulysses,2018-03-05,2018-07-19,2018-03-05 14:22:11,2018-03-05 14:22:11,2,/,130,97.105.19.61
2018-07-13 16:38:29,23,Ulysses,#ulysses,2018-03-05,2018-07-19,2018-03-05 14:22:11,2018-03-05 14:22:11,2,spring,130,97.105.19.61


In [78]:
datascience[datascience.ip == '97.105.19.61']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,


In [79]:
datascience[datascience.ip == '97.105.19.58']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2019-08-20 09:39:58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,466,97.105.19.58
2019-08-20 09:39:59,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,467,97.105.19.58
2019-08-20 09:39:59,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,468,97.105.19.58
2019-08-20 09:40:02,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,469,97.105.19.58
2019-08-20 09:40:08,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,470,97.105.19.58
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-19 07:40:26,55,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,3,/,617,97.105.19.58
2020-03-19 07:40:36,55,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,3,5-stats/1-overview,617,97.105.19.58
2020-03-19 08:32:48,55,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,3,5-stats/2-simulation,617,97.105.19.58


In [80]:
datascience[datascience.ip == '192.171.117.210']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2019-09-05 18:19:21,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,481,192.171.117.210
2019-09-05 18:25:12,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,3-sql/9.4-subqueries,481,192.171.117.210
2019-09-05 18:50:42,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,/,485,192.171.117.210
2019-09-05 18:50:57,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,3-sql/9.3-joins,485,192.171.117.210
2019-09-06 18:07:52,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,3,1-fundamentals/1.1-intro-to-data-science,485,192.171.117.210
...,...,...,...,...,...,...,...,...,...,...,...
2021-03-25 16:38:16,133,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,3,/,841,192.171.117.210
2021-03-25 16:39:07,133,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,3,anomaly-detection/overview,841,192.171.117.210
2021-03-25 16:39:08,133,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,3,anomaly-detection/AnomalyDetectionCartoon.jpeg,841,192.171.117.210


In [81]:
webdev[webdev.ip == '192.171.117.210']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2018-01-26 12:37:53,19,Quincy,#quincy,2017-06-05,2017-09-22,2017-06-05 20:12:10,2017-06-05 20:12:10,1,/,37,192.171.117.210
2018-01-26 12:37:54,19,Quincy,#quincy,2017-06-05,2017-09-22,2017-06-05 20:12:10,2017-06-05 20:12:10,1,mkdocs/search_index.json,37,192.171.117.210
2018-01-26 12:38:12,19,Quincy,#quincy,2017-06-05,2017-09-22,2017-06-05 20:12:10,2017-06-05 20:12:10,1,content/javascript/objects/math.html,37,192.171.117.210
2018-01-26 12:38:19,19,Quincy,#quincy,2017-06-05,2017-09-22,2017-06-05 20:12:10,2017-06-05 20:12:10,1,content/laravel/quickstart/sessions.html,37,192.171.117.210
2018-01-30 12:44:46,14,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,1,/,69,192.171.117.210
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 09:27:31,138,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2,html-css/css-ii/bootstrap-grid-system,938,192.171.117.210
2021-04-21 10:27:35,138,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2,examples/bootstrap-grid/assets.zip,938,192.171.117.210
2021-04-21 12:02:21,138,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2,html-css/css-ii/bootstrap-introduction,938,192.171.117.210


In [83]:
datascience[datascience.ip == '45.56.151.121']

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
